In [ ]:
# Install Python requirements
!python -m pip install -r requirements.txt -Uq

In [1]:
# Authenticate with AWS using your credentials

import os
from dotenv import load_dotenv

# Tải biến môi trường từ file .env
load_dotenv()

# Sử dụng biến môi trường trong code
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_session_token = os.getenv("AWS_SESSION_TOKEN")

In [2]:
# Amazon Bedrock Model ID used throughout this notebook
# Model IDs: https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns
MODEL_ID = "stability.stable-image-ultra-v1:0"

In [3]:
# Location of visual contents used throughout this notebook
SOURCE_IMAGES = "./content/source_images"
SOURCE_VIDEOS = "./content/source_videos"
SOURCE_FRAMES = "./content/source_frames"

GENERATED_IMAGES = "./content/generated_images"
GENERATED_FRAMES = "./content/generated_frames"
GENERATED_VIDEOS = "./content/generated_videos"

In [20]:
import base64
import io
import json
import logging
import boto3
from PIL import Image
import time

from botocore.exceptions import ClientError

# Set up logging
logger = logging.getLogger(__name__)
if logger.hasHandlers():
    logger.handlers.clear()
handler = logging.StreamHandler()
logger.addHandler(handler)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.setLevel(logging.INFO)

# Đường dẫn để lưu hình ảnh được tạo
GENERATED_IMAGES = "./content/generated_images"

class ImageError(Exception):
    """
    Custom exception for errors returned by the model.
    """
    def __init__(self, message):
        self.message = message


def generate_image_from_text(model_id, body):
    """
    Generate an image using the specified model.
    Args:
        model_id (str): The model ID to use.
        body (str): The request body to use.
    Returns:
        image_bytes (bytes): The image generated by the model.
    """

    logger.info("Generating image with model %s", model_id)

    # Khởi tạo client Bedrock với vùng Oregon (us-west-2)
    bedrock = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")

    try:
        # Gửi yêu cầu tới mô hình
        response = bedrock.invoke_model(
            modelId=model_id,
            body=body,
            accept="application/json",
            contentType="application/json"
        )

        # Đọc phản hồi từ mô hình
        response_body = json.loads(response["body"].read().decode("utf-8"))
        # logger.info(f"Bedrock result: {response_body}")

        # Lấy hình ảnh từ phản hồi
        if "images" in response_body and len(response_body["images"]) > 0:
            base64_image = response_body["images"][0]
            image_bytes = base64.b64decode(base64_image)
            logger.info("Successfully generated image with the model %s", model_id)
            return image_bytes
        else:
            raise ImageError("No images found in response.")

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        raise
    except ImageError as err:
        logger.error(err.message)
        raise


def text_to_image_request(
    model_id,
    positive_prompt,
    negative_prompt,
):
    """
    Entrypoint for generating image from text prompt.
    Args:
        model_id (str): The model ID to use.
        positive_prompt (str): The positive prompt to use.
        negative_prompt (str): The negative prompt to use.
    """

    # Xây dựng nội dung yêu cầu (GIỮ NGUYÊN)
    body = json.dumps(
        {
            "prompt": positive_prompt,
            "negative_prompt": negative_prompt,
            "seed": 123456,  # Để đảm bảo tính nhất quán giữa các lần thử nghiệm
            "output_format": "jpeg",  # Định dạng ảnh đầu ra
            "aspect_ratio": "1:1",  # Tỉ lệ khung hình, ở đây là hình vuông
            "mode": "text-to-image",  # 
        }
    )

    # Tạo và lưu hình ảnh
    try:
        image_bytes = generate_image_from_text(model_id=model_id, body=body)
        image = Image.open(io.BytesIO(image_bytes))
        epoch_time = int(time.time())
        generated_image_path = f"{GENERATED_IMAGES}/image_{epoch_time}.jpg"
        logger.info(f"Generated image: {generated_image_path}")
        image.save(generated_image_path, format="JPEG", quality=95)
    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
    except ImageError as err:
        logger.error(err.message)
    else:
        logger.info(f"Finished generating image with the model {model_id}.")

In [19]:
# There is a 77-token limit for prompts for the SXDL 1.0 model

# Phoenix prompt inspiration: https://civitai.com/images/3030250
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 25
# POSITIVE_PROMPT = "mythical beast, Phoenix, big eyes, sharp beak, long flowing feathers, bright colors, outdoors, perched on branch, no_humans, fire,scenery, cinematic lighting, strong contrast, high level of detail, best quality, masterpiece, best quality, ultra-detailed, masterpiece, hires, 8k"
# NEGATIVE_PROMPT = "(worst quality:1.5), (low quality:1.5), (normal quality:1.5), lowres, bad anatomy, bad hands, watermark, moles, toe, bad-picture-chill-75v, realisticvision-negative-embedding, (monochrome:1.5), (grayscale:1.5), (bad proportions:1.3)"

# Samurai prompt inspiration: https://prompthero.com/prompt/accdc184a10
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 40
# POSITIVE_PROMPT = "digital wallpaper portrait of a samurai in black armor, highly detailed, at night, dramatic cinematic lighting, rainy, reflections in puddles, dramatic composition, emotionally profound, leaving an indelible and haunting impression on psyche, very intricate, by Milo Manara and Russ Mills"
# NEGATIVE_PROMPT = "(deformed iris, deformed pupils), text, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, (extra fingers:1.5), (mutated hands:1.5), poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, (fused fingers), (too many fingers), long neck, camera, name, signature, watermark, logo, autograph, trademark, cut off, censored, bad anatomy, bad body, bad face, bad teeth, deformities, (boring, uninteresting:1.1)"

# Cat prompt inspiration: https://civitai.com/images/6826034
# No preset style, cfg_scale: 12, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 25
# POSITIVE_PROMPT = "amazing quality, masterpiece, best quality, hyper detailed, ultra detailed, UHD, perfect anatomy, magic world, (kitten and fish:1.4), fish in the air, spell magic to get fresh fish as food,( fish jumping from magic book:1.3), energy flow, a full body of a cute kitten, kawaii, wearing witches robe, witches hat, holding magic book, magic book on one hand, spell magic, hkmagic, extremely detailed, glowneon, glowing"
# NEGATIVE_PROMPT = "FastNegativeV2, watermark, signature, worst quality, low quality, normal quality, lowres, simple background, inaccurate limb, extra fingers, fewer fingers, missing fingers, extra arms, (extra legs:1.3), inaccurate eyes, bad composition, bad anatomy, error, extra digit, fewer digits, cropped, low res, worst quality, low quality, normal quality, jpeg artifacts, extra digit, fewer digits, trademark, watermark, artist's name, username, signature, text, words"

# Robot prompt inspiration: https://civitai.com/images/6935869
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 40
# POSITIVE_PROMPT = "masterpiece, best quality, cinematic film still, realistic, portrait, solo, white mecha robot, cape, science fiction, torn clothes, glowing, standing, robot joints, mecha, armor, cowboy shot, (floating cape:1.3), intense sunlight, silver dragonborn, outdoors, landscape, nature highres, 4k, 8k, intricate detail, cinematic lighting, amazing quality, wallpaper"
# NEGATIVE_PROMPT = "nipples, (low quality, worst quality:1.4), cgi, text, signature, watermark, extra limbs"

# Batman prompt inspiration: https://medium.com/phygital/top-40-useful-prompts-for-stable-diffusion-xl-008c03dd0557
# No preset style, cfg_scale: 5, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 35
# POSITIVE_PROMPT = "batman, cute modern disney style, Pixar 3d portrait, ultra detailed, gorgeous, 3d zbrush, trending on dribbble, 8k render"
# NEGATIVE_PROMPT = ""

# Dragon prompt inspiration: https://civitai.com/images/7029232
# No negative prompt, preset style: anime, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 1121313, steps: 25
# POSITIVE_PROMPT = "silk screen art, (ukiyo-e:1.1), (painted by Kim Hong-do:1.1), sharp oriental dragon, green and gold with a bit of blue, katsuhiro otomo, line art, glitch art, hergé"
# NEGATIVE_PROMPT = ""

# Autumn girl prompt inspiration: https://civitai.com/images/6840704
# v1 No negative prompt, no preset style, cfg_scale: 5, sampler: K_DPMPP_2M, samples: 1, seed: 123456, steps: 30
# v2 No negative prompt, no preset style, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 123456, steps: 25
# POSITIVE_PROMPT = "mysterious silhouette forest woman, by Minjae Lee, Carne Griffiths, Emily Kell, Geoffroy Thoorens, Aaron Horkey, Jordan Grimmer, Greg Rutkowski, amazing depth, masterwork, surreal, geometric patterns, intricately detailed, bokeh, perfect balanced, deep fine borders, artistic photorealism , smooth, great masterwork by head of prompt engineering"
# NEGATIVE_PROMPT = "boring,text,signature,logo,watermark,low quality, bad quality, loose artifacts, grainy, blurry, long neck, closed eyes, face jewellery"

text_to_image_request(
    MODEL_ID,
    POSITIVE_PROMPT,
    NEGATIVE_PROMPT,
)

2024-10-11 01:00:35,166 - INFO - Generating image with model stability.stable-image-ultra-v1:0
2024-10-11 01:00:50,056 - INFO - Bedrock result: {'seeds': [123456], 'finish_reasons': [None], 'images': ['...']}
2024-10-11 01:00:50,063 - INFO - Successfully generated image with the model stability.stable-image-ultra-v1:0
2024-10-11 01:00:50,131 - INFO - Generated image: ./content/generated_images/image_1728583250.jpg
2024-10-11 01:00:50,154 - INFO - Finished generating image with the model stability.stable-image-ultra-v1:0.


In [ ]:
# Display an example of a generated image

from PIL import Image
from IPython.display import display

# SOURCE_IMAGE = f"{GENERATED_IMAGES}/text_to_image_samples/phoenix_1.jpg"
SOURCE_IMAGE = f"{GENERATED_IMAGES}/image_1728542205.jpg"
source_image = Image.open(SOURCE_IMAGE).resize((384, 384))
display(source_image)
print(SOURCE_IMAGE)